In [1]:
import findspark
findspark.init()
import pyspark
sc = pyspark.SparkContext(appName="")
import re
import random

In [2]:
## Note that to I have excluded the licensen information from the downloaded for text for better accuracy
## Pre-process the text to create a list of words without all caps, all caps with an ending period, and all nums
llist = sc.textFile('source.txt')
wlist = llist.flatMap(lambda x: x.split())
wlist_filtered = wlist.filter(lambda x: (not x[:-1].isupper()) if x[-1] == '.' else ((not x.isupper()) and (not x.isdigit())))

In [3]:
## With the cleaned word list, index, offset and then join the list(s) to create (w1,w2),[(w3,w3_count),...]
wlist_index = wlist_filtered.zipWithIndex().map(lambda x: (x[1], x[0]))
wlist_offset1 = wlist_index.map(lambda x: (x[0]-1, x[1]))
wlist_offset2 = wlist_offset1.map(lambda x: (x[0]-1, x[1]))
wlist_join = wlist_index.join(wlist_offset1).join(wlist_offset2)
word_set = wlist_join.map(lambda x: x[1]).groupByKey().map(lambda (k,v): (k, list(v))).map(lambda (k, v): (k, [(x, v.count(x)) for x in set(v)])).cache()

In [4]:
## According the the 2 previous words, randomly choose the 3rd word weighted by the word count in the "dictionary"
## This is done by randomly selecting an item from a list of [word] * word_count
## Then we print the 2 sample key + 18 generated words
def sentence(num_sentences, num_words, dictionary):
    for i in range(num_sentences):
        sample = dictionary.takeSample(True, 1)
        sentence = [w for w in sample[0][0]]
        for j in range(2,num_words):
            next_choice_raw = dictionary.map(lambda x: x).lookup((sentence[-2], sentence[-1]))
            next_word = random.choice([w for wl in [[x[0]]*x[1] for x in list(*next_choice_raw)] for w in wl])
            sentence.append(next_word)
            sentence2 = ' '.join(sentence) + '\n'
        print sentence2

sentence(10, 20, word_set)

but Ay, we may Taste of it Freely command, thou wouldst fly death. have words to them. Would you, in

monster's, wants not minister to love, That like the adder And that you'll say ere half an hour, One minute,

Poins and will do none to-morrow: he will trouble you no modesty, no maiden shame, No touch of consanguinity, No

falsehood, 'As false as Cressid.' Go to, go to; you have shown all Hectors. Enter the Archbishop of Canterbury. Come,

of Macbeth; marry, he was his mistress. Well, and say 'Ay.' And yet, methinks, could despise this man, for keeping

mine! But virtue, as it was not to stop my nose; or against any denial. Tell him he hath simply

the adverse side, should not be ransom for offence, tender 't here; do as we. My father charg'd you in

Clifford, ask mercy and remorse, Cool and congeal again to what sport and pleasure Than that a lady richly left,

for ten shillings. Prince. Well then, France shall rue. Farewell, my Hector and Achilles meet; For both our armies here,

ea